In [1]:
#import preprocessed per user dataset from onedrive prickled file
%load_ext autotime
import pickle
import os
import numpy as np

path = r"C:/Users/natha/OneDrive - The University of Sydney (Students)/CS48-CAPSTONE Project 2021 Sem1/dataset"
#create output dictionary
#enter number of user here
N_user = 1000

data = {}
for u in os.listdir(path+'/user_preprocessed_pickle')[:N_user]: #import N users

    uid = int(u[:-7])
    file_name = '/'.join([path,'/user_preprocessed_pickle', u])

    try:
        with open(file_name, 'rb') as f:
            dic = pickle.load(f)
            data[uid] = dic[uid]
    except:
        print(dic)
        
total_user = len(data.keys())
print("Imported {} users".format(total_user))

time: 6.62 s


In [2]:
#create 2 function to extract the feature and label from array
def extract_feature(lst):
    #extract first 2 columns: session and mode location 
    return [item[:2] for item in lst]

def extract_label(lst):
    #extract the remaning columns
    return [item[2:] for item in lst]

# function to generate input for baseline model
def generate_baseline_input(data, mode):
    data_neural = data
    data_feature = []
    data_label = []

    if mode == 'train':
        day_id = [20, 21, 22, 25] #the day for training

    elif mode == 'test':
        day_id = [26] # the day for testing, should remove 26 which is left for real testing

    # if candidate is None:
    candidate = data_neural.keys() #filter, and get user id

    #iterate all the users
    for u in candidate:
        #seperate feature and label list
        user_X_train = []
        user_y_label = []
        #get user's record
        sessions = data_neural[u]
        #seperate days for training and testing
        for i in day_id:
            #call specific day
            session = data_neural[u][i]
            #append feature to feature list, label to label list
            # user_feature.append(extract_feature(session))
            # user_label.append(extract_label(session))
            timeloc = extract_feature(session)[1:]
            prev_app_count = extract_label(session)[:-1]
            target = extract_label(session)[1:]
            user_feature = np.concatenate((timeloc,prev_app_count),axis = 1)
            user_X_train.append(user_feature)
            user_y_label.append(target)
        #after iterate the day, need to shift the feature columns upwards for prediction
        #also need to cut the last rows for labels
        #reshape: as orginal dataset were seperate by day,session,column:
        #feature:4(days),48sessions,2features
        #label:4(days),48sessions,2000 app usage(count)
        #reshape to flattern the data
        user_X_train = np.array(user_X_train).reshape(-1,2002)
        user_y_label = np.array(user_y_label).reshape(-1,2000)
        #append per user's feature and label to all_user list
        data_feature.append(user_X_train)
        data_label.append(user_y_label)

    data_feature = np.array(data_feature)
    data_label = np.array(data_label)

    return data_feature, data_label


time: 0 ns


In [3]:
X_train, y_train = generate_baseline_input(data,'train')
print(X_train.shape)
print(y_train.shape)
#n users, n rows per user , 2 features/2000 labels


(870, 188, 2002)
(870, 188, 2000)
time: 5.33 s


In [4]:
# X_train, y_train = generate_input(data, 'train')
# reshape to merge user
X_train = X_train.reshape(-1,2002)
y_train = y_train.reshape(-1,2000)

print(X_train.shape)
print(y_train.shape)

(163560, 2002)
(163560, 2000)
time: 0 ns


In [5]:
X_test, y_test = generate_baseline_input(data, 'test')
X_test = X_test.reshape(-1,2002)
y_test = y_test.reshape(-1,2000)
print(X_test.shape)
print(y_test.shape)

(40890, 2002)
(40890, 2000)
time: 1.12 s


In [6]:
# onehot encode the labels (turn count to one hot > used/not used)
y_train = np.where(y_train > 0, 1, 0)
y_test = np.where(y_test > 0, 1, 0)

time: 1.19 s


# Naive Bayes Model

In [15]:
#user the MultiOutputClassifier to fit one classifier to one target (so every rows of lable are trained with a seperate model)
#pick Naive bayes purely because it is fast to train

from sklearn.naive_bayes import BernoulliNB
from sklearn.multioutput import MultiOutputClassifier

clf_BNB = MultiOutputClassifier(BernoulliNB()).fit(X_train, y_train)
# pred_BNB = clf_BNB.predict(X_test)
# pred_BNB_prob = clf_BNB.predict_proba(X_test)
# need the predict_proba to generate prob for each class(count)
#as one hot coded it is just 2 class(0 for not use, 1 for use)

In [16]:
pred_BNB = clf_BNB.predict(X_test)

In [17]:
pred_BNB_prob = clf_BNB.predict_proba(X_test)

In [8]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.multioutput import MultiOutputClassifier

# clf_MNB = MultiOutputClassifier(MultinomialNB()).fit(X_train, y_train)
# pred_MNB = clf_MNB.predict(X_test)
# pred_MNB_prob = clf_MNB.predict_proba(X_test)

In [18]:
#if train by count (Not one hot coded) might need this function to extract the probabiity of class 0, then 1- for the probability of using the app in next session

def extract_prob(lst):
    #first elements is prob for class 0, extract prob class 0 then 1- prob class 0 = prob class 1
    
    return [(1-item[0]) for item in lst]

for i in range(2000):
    pred_BNB_prob[i] =  extract_prob(pred_BNB_prob[i])

#Transpose to match the shape
pred_BNB_prob = np.array(pred_BNB_prob).T
pred_BNB_prob.shape

(40890, 2000)

# MLR

In [9]:
all0_column_ind = np.argwhere(np.all(y_train == 0, axis = 0))
print(len(all0_column_ind))

378
time: 281 ms


In [ ]:
y_train_rm = np.delete(y_train, all0_column_ind, axis=1)
y_test_rm = np.delete(y_test, all0_column_ind, axis=1)

# MLP (Ignore this part for now)

In [34]:
#just random picked one model for testing, should have a lot of models
from sklearn.multioutput import MultiOutputClassifier
# from sklearn.neighbors import KNeighborsClassifier ##/
# from sklearn.linear_model import LogisticRegression ##X
from sklearn.neural_network import MLPClassifier

clf_tanh = MultiOutputClassifier(MLPClassifier(random_state=1, max_iter=300, activation = 'tanh')).fit(X_train, y_train)
pred_tanh_prob = clf_tanh.predict_proba(X_test)
pred_tanh = clf_tanh.predict(X_test)

In [37]:
#predict_proba > return the probability of every class 
#here class means the count occurs in train set
#all we want is the prob of class 0, so we can 1- prob class 0 for class 1

def extract_prob(lst):
    return [(1-item[0]) for item in lst]

for i in range(2000):
    pred_tanh_prob[i] =  extract_prob(pred_tanh_prob[i])

pred_tanh_prob = np.array(pred_tanh_prob).T
pred_tanh_prob.shape

(4700, 2000)

# Evaluation
## Follows the same evaluation metrics with DeepApp

In [47]:
# all_acc_session = []
# y_test = y_test.astype(np.int64)
# pred = pred_MNB.astype(np.int64)

# for i in range(y_test.shape[0]):
#     correct_count = 0
#     pred_app_session = [app_ind for app_ind,app_count in enumerate(pred[i]) if app_count !=0]
#     true_app_session = [app_ind for app_ind,app_count in enumerate(y_test[i]) if app_count !=0]
#     if ((len(true_app_session) == 0) & (len(pred_app_session) == 0)):
#         all_acc_session.append(1)
#     else:
#         for j in pred_app_session:
#             if j in true_app_session:
#                 correct_count +=1
#         acc = correct_count/len(true_app_session)
#         all_acc_session.append(acc)

# print("Precision MNB: {:.5f}".format(np.mean(all_acc_session)))

Precision MNB: 0.83533


In [19]:
def cal_ap( y_actual, y_pred, k ):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # Calculation of precision
    num = 0
    rank = 0
    sumP = 0.0
    for item in s_zip_topk:
        rank += 1
        if item[0] == 1:
            num += 1
            sumP += (num*1.0)/(rank*1.0)
    ap = 0.0
    if num > 0:
        ap = sumP/(num*1.0)
    return ap   # average precision
# Take topk prediction and the ground truth


def r_k(y_actual, y_pred, k, threshold):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # print(s_zip_topk)
    actual, pred = zip(*s_zip_topk)
    actual = np.where(np.array(actual) > threshold, 1, 0)
    pred = np.array(pred)
    pred_o = np.where(pred > threshold, 1, 0)
    return actual, pred_o, pred

# acc[3] += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5

# Naive Bayes

In [31]:
from sklearn import metrics as skmetrics

avg_auc = 0
avg_map = 0
avg_recall = 0
count = 0

for i in range(y_test.shape[0]):
    if (np.sum(y_test[i])> 0):
        fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], pred_BNB_prob[i], pos_label=1)
        avg_auc += skmetrics.auc(fpr, tpr)
        actual, pred_o, pred = r_k(y_test[i], pred_BNB_prob[i] ,5, 5)
        avg_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5
        avg_map += cal_ap(y_test[i], pred_BNB_prob[i], 5)
    else:
        count +=1
        pass
    
print('AUC: ', avg_auc / y_test.shape[0])
print('MAP: ', avg_map / y_test.shape[0])
print('Recall@5: ', avg_recall / y_test.shape[0])
print('Skipped: {} rows, total: {} rows'.format(count, y_test.shape[0]))

# fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# acc[0] += skmetrics.auc(fpr, tpr)
# # acc[1] += cal_ap(truth, predict, 5)

AUC:  0.26008215917912597
MAP:  0.13543381973315738
Recall@5:  0.279334800684764
Skipped: 29468 rows, total: 40890 rows


In [32]:
# from sklearn import metrics as skmetrics

# avg_auc = 0
# avg_map = 0
# avg_recall = 0
# count = 0

# for i in range(y_test.shape[0]):
#     if np.sum(pred_BNB_prob[i])> 0:
#         fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], pred_BNB_prob[i], pos_label=1)
#         avg_auc += skmetrics.auc(fpr, tpr)
#         actual, pred_o, pred = r_k(y_test[i], pred_BNB_prob[i] ,5, 5)
#         avg_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5
#         avg_map += cal_ap(y_test[i], pred_BNB_prob[i], 5)
#     else:
#         count +=1
#         pass
    
# print('AUC: ', avg_auc / y_test.shape[0])
# print('MAP: ', avg_map / y_test.shape[0])
# print('Recall@5: ', avg_recall / y_test.shape[0])

# print('{} rows has been skipped'.format(count))

# # fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# # acc[0] += skmetrics.auc(fpr, tpr)
# # # acc[1] += cal_ap(truth, predict, 5)

# MLP

In [39]:
from sklearn import metrics as skmetrics

avg_auc = 0
avg_map = 0
avg_recall = 0
for i in range(2000):
    if np.sum(y_test[i])> 0:
        fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], pred_tanh_prob[i], pos_label=1)
        avg_auc += skmetrics.auc(fpr, tpr)
        actual, pred_o, pred = r_k(y_test[i], pred_tanh_prob[i] ,5, 5)
        avg_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5
        avg_map += cal_ap(y_test[i], pred_tanh_prob[i], 5)
    else:
        pass
print('AUC:', avg_auc / 4700)
print('MAP', avg_map / 4700)
print('Recall@5', avg_recall / 4700)

# fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# acc[0] += skmetrics.auc(fpr, tpr)
# # acc[1] += cal_ap(truth, predict, 5)

AUC: 0.23330724614377835
MAP 0.11360763888888885
